
<div style="text-align: center; padding: 30px 10px;">

<h1 style="color:#ff7500; font-size: 24px; margin-bottom: 10px;">
МФТИ ФПМИ
</h1>

<h2 style="font-size: 30px; margin-top: 5px;">
Практикум Python - Продвинутый Поток
</h2>

<hr style="width: 60%; border: 1px solid #10069f; margin: 25px auto;">

<h3 style="font-size: 36px;">
4. Итераторы и генераторы
</h3>

<p style="margin-top: 20px;">
<strong>Дата:</strong> 23-28 февраля 2026 года<br>
</p>

<p style="margin-top: 25px;">
Данный ноутбук является частью серии семинаров по курсу  
<em>«Практикум Python»</em> и предназначен для учебных и образовательных целей.
</p>

</div>

# Итераторы

In [ ]:
for i in range(5):
    pass

for line in open('file.txt'):
    pass

for key in {'A' : 1, 'B' : 2, 'C' : 3}:
    pass

for letter in 'Hello, World':
    pass

In [18]:
iterable = [1, 2, 3]
iterator = iterable.__iter__()
iterator

In [19]:
iterator.__next__()

1

In [20]:
iterator.__next__()

2

In [21]:
iterator.__next__()

3

In [22]:
iterator.__next__()

StopIteration: 

### [Iterable](https://docs.python.org/3/glossary.html#term-iterator)

Это объект, у которого определён метод `__iter__`,

возвращающий **итератор**.

Примеры: `list`, `dict`, `range`

### [Iterator](https://docs.python.org/3/glossary.html#term-iterator)

Это объект, у которого определены методы `__iter__` и `__next__`.

Метод `__iter__` должен возвращать сам итератор (`self`).

Метод `__next__` должен возвращать следующий элемент,

а если их не осталось, выкидывать исключение  `StopIteration`.

## [iter](https://docs.python.org/3/library/functions.html#iter) & [next](https://docs.python.org/3/library/functions.html#next)

In [24]:
iterator = iter([1])
iterator

In [25]:
next(iterator)

1

In [26]:
next(iterator)

StopIteration: 

In [27]:
next(iterator, 'some value')

'some value'

## Вторая форма оператора iter

In [28]:
import io

stream = io.StringIO('abcdefghi')

def read3() -> str:
    return stream.read(3)

In [29]:
iter(read3, '')  # every __next__ translates to __call__

In [30]:
for chunk in iter(read3, ''):  # iter(callable, sentinel)
    print(chunk, end=' ')

abc def ghi 

In [31]:
from collections.abc import Callable

def make_timer(ticks: int) -> Callable[[], int]:

    def timer() -> int:
        nonlocal ticks
        ticks -= 1
        return ticks

    return timer

In [32]:
timer = make_timer(2)

In [33]:
timer()

1

In [34]:
timer()

0

In [35]:
for i in iter(make_timer(10), -1):
    print(i, end=' ')

9 8 7 6 5 4 3 2 1 0 

## Реализация цикла for через while

In [36]:
for value in sequence:
    ...

NameError: name 'sequence' is not defined

In [37]:
iterator = iter(sequence)
while True:
    try:
        value = next(iterator)
    except StopIteration:
        break
    else:
        ...

NameError: name 'sequence' is not defined

## О хранении итератором состояния

In [38]:
iterable = range(10)
iterable

range(0, 10)

In [39]:
iterator = iter(iterable)
iterator

In [40]:
zip_iterator = zip(iterator, iterator)
zip_iterator

In [41]:
for pair in zip_iterator:
    print(pair, end=' ')

(0, 1) (2, 3) (4, 5) (6, 7) (8, 9) 

## Объект и его итератор

In [42]:
iterable = open('IteratorsGenerators.ipynb')
iterable

FileNotFoundError: [Errno 2] No such file or directory: 'IteratorsGenerators.ipynb'

In [43]:
iter(iterable)

In [44]:
iterable is iter(iterable)

False

In [45]:
class TextIOWrapper:
    ...

    def __iter__(self) -> 'TextIOWrapper':
        return self

    ...

In [46]:
class FilelikeRange:
    def __init__(self, start: int, stop: int) -> None:
        self._index = start
        self._stop = stop

    def __iter__(self) -> 'FilelikeRange':
        return self

    def __next__(self) -> int:
        if self._index >= self._stop:
            raise StopIteration()
        value = self._index
        self._index += 1
        return value

In [47]:
for i in FilelikeRange(0, 5):
    print(i, end=' ')

0 1 2 3 4 

In [48]:
iterable = [1, 2, 3, 4, 5]
iterable

[1, 2, 3, 4, 5]

In [49]:
iter(iterable)

In [50]:
iterable is iter(iterable)

False

In [51]:
class ListlikeRange:
    class Iterator:
        def __init__(self, start: int, stop: int) -> None:
            self._index = start
            self._stop = stop

        def __next__(self) -> int:
            if self._index >= self._stop:
                raise StopIteration()
            value = self._index
            self._index += 1
            return value

    def __init__(self, start: int, stop: int) -> None:
        self._start = start
        self._stop = stop

    def __iter__(self) -> Iterator:
        return self.Iterator(self._start, self._stop)

In [52]:
for i in ListlikeRange(0, 5):
    print(i, end=' ')

0 1 2 3 4 

## Истощаемость

In [53]:
filelike_range = FilelikeRange(1, 5)
listlike_range = ListlikeRange(1, 5)

In [54]:
for elem in filelike_range:
    print(elem, end=' ')

1 2 3 4 

In [55]:
for elem in filelike_range:
    print(elem, end=' ')

In [56]:
for elem in listlike_range:
    print(elem, end=' ')

1 2 3 4 

In [57]:
for elem in listlike_range:
    print(elem, end=' ')

1 2 3 4 

## [Sequence](https://docs.python.org/3/glossary.html#term-sequence) как iterable

In [58]:
from typing import TypeVar

T = TypeVar('T')

class Sequence:
    def __init__(self, *args: T) -> None:
        self._args = args

    def __len__(self) -> int:
        return len(self._args)

    def __getitem__(self, index: int) -> T:
        if index < 0 or index >= len(self):
            raise IndexError(index)  # expected by for to detect eos
        return self._args[index]

In [59]:
seq = Sequence(1, 2, 3, 4, 5)
seq[0], seq[2], seq[4]

(1, 3, 5)

In [60]:
for i in seq:
    print(i, end=' ')

1 2 3 4 5 

## [\_\_contains__](https://docs.python.org/3/reference/datamodel.html#object.__contains__)

In [61]:
3 in range(5)

True

In [62]:
# https://docs.python.org/3.11/reference/expressions.html#membership-test-details
# default __contains__ looks like
def __contains__(self, value: Any) -> bool:
    for item in self:
        if item is value or item == value:
            return True
    return False

In [63]:
class MyRange:
    def __contains__(self, value: int) -> bool:
        return 0 <= value < self._stop

    ...

In [64]:
seq = Sequence(2, 3, 5, 8, 13, 21)

In [65]:
for i in seq:
    print(i, end=' ')

2 3 5 8 13 21 

In [66]:
8 in seq  # object has no __contains__, so "in" uses iteration over __getitem__

True

### Некоторые функции для работы с итераторами

### [enumerate](https://docs.python.org/3/library/functions.html#enumerate)

In [67]:
for i, char in enumerate('sample'):  # enumerate(iterable, start=index)
    print(i, char)

0 s
1 a
2 m
3 p
4 l
5 e


### [zip](https://docs.python.org/3/library/functions.html#zip)

In [68]:
for left, right in zip('ABCD', 'xy'):
    print(left + right)

Ax
By


In [69]:
from itertools import zip_longest

for left, right in zip_longest('ABCD', 'xy', fillvalue='-'):
    print(left + right)

Ax
By
C-
D-


### Обращаем zip

In [70]:
x = [1, 2, 3]
y = [4, 5, 6]
zipped = zip(x, y)

In [71]:
for item in zipped:
    print(item, end=' ')

(1, 4) (2, 5) (3, 6) 

In [72]:
zipped = zip(x, y)

In [73]:
x2, y2 = zip(*zipped)
print(x2, y2)

(1, 2, 3) (4, 5, 6)


### [map](https://docs.python.org/3/library/functions.html#map) & [filter](https://docs.python.org/3/library/functions.html#filter)

In [74]:
for squared in map(lambda x: x ** 2, range(5)):
    print(squared, end=' ')

0 1 4 9 16 

In [75]:
for filtered in filter(lambda x: x % 2 == 0, range(10)):
    print(filtered, end=' ')

0 2 4 6 8 

### [itertools](https://docs.python.org/3/library/itertools.html) & [more](https://more-itertools.readthedocs.io/en/stable/)

### [itertools.chain](https://docs.python.org/3/library/itertools.html#itertools.chain)

In [76]:
from itertools import chain

In [77]:
for elem in chain(range(5), [10, 20], 'sample', [[i] for i in range(5)]):
    print(elem, end=' ')

0 1 2 3 4 10 20 s a m p l e [0] [1] [2] [3] [4] 

In [78]:
from typing import Any

def repeat(times: int, obj: Any) -> list[Any]:
    return [obj] * times

In [79]:
list(repeat(5, [1, 2, 3]))

[[1, 2, 3], [1, 2, 3], [1, 2, 3], [1, 2, 3], [1, 2, 3]]

In [80]:
list(chain.from_iterable(repeat(5, [1, 2, 3])))

[1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3, 1, 2, 3]

### Как проверить объект на итерируемость (с гарантией)

In [81]:
try:
    iter(object_to_test)
except TypeError:
    # not an iterable
    ...
else:
    # iterable
    ...

NameError: name 'object_to_test' is not defined

### [itertools.tee](https://docs.python.org/3/library/itertools.html#itertools.tee)

In [82]:
from itertools import tee

![tee](http://4.bp.blogspot.com/-u_KYBwIUyF4/UUR5cvbv6PI/AAAAAAAAAXs/hPJT0ZR5iBc/s1600/tee_diagram.png)

In [83]:
iterator1, iterator2 = tee(range(3), 2)

In [84]:
for elem in iterator1:
    print(elem, end=' ')

for elem in iterator2:
    print(elem, end=' ')

0 1 2 0 1 2 

### [itertools.groupby](https://docs.python.org/3/library/itertools.html#itertools.groupby)

In [85]:
from itertools import groupby

In [86]:
for key, group in groupby('AABBCCDAAB'):
    print(key, list(group))

A ['A', 'A']
B ['B', 'B']
C ['C', 'C']
D ['D']
A ['A', 'A']
B ['B']


In [87]:
words = ['cab', 'face', 'cafe', 'abc', 'goo']
words = sorted(words, key=sorted)
words

['cab', 'abc', 'face', 'cafe', 'goo']

In [88]:
for key, group in groupby(words, key=sorted):
    print(','.join(key), list(group))

a,b,c ['cab', 'abc']
a,c,e,f ['face', 'cafe']
g,o,o ['goo']


# Генераторы

In [89]:
from collections.abc import Iterator

def countdown(n: int) -> Iterator[int]:
    print(f'Counting down from {n}')
    for i in range(n, 0, -1):
        yield i
    print('Done')

In [90]:
for i in countdown(5):
    print(i)

Counting down from 5
5
4
3
2
1
Done


In [91]:
countdown

<function __main__.countdown(n: int) -> collections.abc.Iterator[int]>

In [92]:
counter = countdown(10)
counter

<generator object countdown at 0x7c02519151c0>

In [93]:
iter(counter) is counter

True

In [94]:
counter = countdown(2)

In [95]:
next(counter)

Counting down from 2


2

In [96]:
next(counter)

1

In [97]:
next(counter)

Done


StopIteration: 

### [Generator](https://docs.python.org/3/glossary.html#term-generator)

Это специальный итератор, который получается

в результате вызова функции, содержащей ключевое слово `yield`.

Последовательность значений, которую возвращает генератор,

задается последовательностью операторов `yield` в теле функции.

### Примеры

In [98]:
def squares(size: int) -> Iterator[int]:
    for i in range(size):
        yield i ** 2

In [99]:
generator = squares(5)

In [100]:
for elem in generator:
    print(elem, end=' ')

0 1 4 9 16 

In [101]:
for elem in generator:
    print(elem, end=' ')

Генераторы истощаются!

In [102]:
from collections.abc import Iterable, Iterator
from typing import TypeVar

T = TypeVar('T')

def unique_ordered(elements: Iterable[T]) -> Iterator[T]:
    seen = set()
    for elem in elements:
        if elem in seen:
            continue
        seen.add(elem)
        yield elem

In [103]:
for elem in unique_ordered([1, 2, 3, 1, 2, 4]):
    print(elem, end=' ')

1 2 3 4 

### Цепочка генераторов

In [104]:
def sum_of_squares_of_even(iterable: Iterable[int]) -> int:
    sum_ = 0
    for i in iterable:
        if i % 2 != 0:
            continue
        sum_ += i ** 2
    return sum_

In [105]:
sum_of_squares_of_even(range(10))

120

In [106]:
def even(iterable: Iterable[int]) -> list[int]:
    result = []
    for i in iterable:
        if i % 2 != 0:
            continue
        result.append(i)
    return result

In [107]:
def squares(iterable: Iterable[int]) -> list[int]:
    result = []
    for i in iterable:
        result.append(i ** 2)
    return result

In [108]:
sum(squares(even(range(10))))

120

In [109]:
def even(iterable: Iterable[int]) -> Iterator[int]:
    for elem in iterable:
        if elem % 2 == 0:
            yield elem

In [110]:
def squares(iterable: Iterable[int]) -> Iterator[int]:
    for elem in iterable:
        yield elem ** 2

In [111]:
sum(squares(even(range(10))))

120

Цепочка генераторов позволяет легко декомпозировать алгоритм

без существенных затрат памяти.

### Генераторные выражения ([generator expression](https://docs.python.org/3/glossary.html#term-generator-expression))

In [112]:
squares = (x ** 2 for x in range(5))
squares

<generator object <genexpr> at 0x7c0251947c60>

In [113]:
for square in squares:
    print(square, end=' ')

0 1 4 9 16 

In [114]:
max(x for x in range(10_000_000_000) if x % 11 == 0)

KeyboardInterrupt: 

In [115]:
max([x for x in range(10_000_000_000) if x % 11 == 0])  # ~20G RAM

KeyboardInterrupt: 

In [116]:
import sys

int_size_bytes = sys.getsizeof(0)
int_count = 10_000_000_000 / 11
list_size_bytes = int_size_bytes * int_count
list_size_gigabytes = list_size_bytes / (1024 ** 3)
list_size_gigabytes

23.706392808394

### Генераторы в качестве итераторов

In [117]:
from collections.abc import Iterator
from dataclasses import dataclass

@dataclass
class BinaryTreeNode:
    value: int
    left: 'BinaryTreeNode | None' = None
    right: 'BinaryTreeNode | None' = None

    def __iter__(self) -> Iterator[int]:  # in-order
        for value in (self.left or ()):
            yield value

        yield self.value

        for value in (self.right or ()):
            yield value

In [118]:
tree = BinaryTreeNode(
    left=BinaryTreeNode(
        left=BinaryTreeNode(value=1),
        value=2,
    ),
    value=3,
    right=BinaryTreeNode(
        value=4,
        right=BinaryTreeNode(value=5),
    ),
)

In [119]:
for value in tree:
    print(value, end=' ')

1 2 3 4 5 

### [yield как выражение](https://docs.python.org/3/reference/simple_stmts.html#yield)

![yield-expr](https://i0.wp.com/storage.googleapis.com/ssivart/super9-blog/priming-generator.png?w=1200&ssl=1)

In [124]:
from typing import Generator

def jumping_counter(upto: int) -> Generator[int, int, None]:
    count = 1
    while count <= upto:
        jump = yield count
        count += jump or 1

In [125]:
generator = jumping_counter(3)

In [126]:
next(generator)  # equals to .send(None)

1

In [127]:
generator.send(2)

3

In [128]:
next(generator)

StopIteration: 

### [throw](https://docs.python.org/3/reference/expressions.html#generator.throw)

In [129]:
generator = jumping_counter(5)

In [130]:
next(generator)

1

In [131]:
generator.throw(Exception('Good luck!'))

Exception: Good luck!

### [close](https://docs.python.org/3/reference/expressions.html#generator.close)

In [132]:
generator = jumping_counter(5)

In [133]:
next(generator)

1

In [134]:
generator.close()

In [135]:
next(generator)

StopIteration: 

### Обработка close

In [136]:
def create_generator() -> Iterator[int]:
    while True:
        try:
            yield 42
        except GeneratorExit:  # can't be ignored
            print('Exiting...')
            return

In [137]:
generator = create_generator()

In [138]:
next(generator)

42

In [139]:
generator.close()

Exiting...


### [@contextmanager](https://docs.python.org/3/library/contextlib.html#contextlib.contextmanager)

In [140]:
from contextlib import contextmanager
import tempfile
import shutil

In [141]:
@contextmanager
def tempdir():
    dirname = tempfile.mkdtemp()
    try:
        yield dirname
    finally:
        shutil.rmtree(dirname)

In [142]:
with tempdir() as path:
    print(path)

/tmp/tmpmfl4w2rd


In [143]:
# https://github.com/python/cpython/blob/b3f0ceae919c1627094ff628c87184684a5cedd6/Lib/contextlib.py#L142

class _GeneratorContextManager:
    def __init__(self, func, args, kwargs):
        self.gen = func(*args, **kwargs)

    def __enter__(self):
        return next(self.gen)

    def __exit__(self, exc_type, exc_value, exc_traceback):
        if exc_type is None:
            try:
                next(self.gen)
            except StopIteration:
                return False
            raise RuntimeError("generator didn't stop")
        else:
            try:
                self.gen.throw(exc_type, exc_value, exc_traceback)
            except BaseException:
                return False
            raise RuntimeError("generator didn't stop after throw()")

def contextmanager(func):
    def helper(*args, **kwargs):
        return _GeneratorContextManager(func, args, kwargs)
    return helper

### [yield from](https://peps.python.org/pep-0380/)

In [144]:
from collections.abc import Iterable
from typing import TypeVar

T = TypeVar('T')

def repeat(times: int, iterable: Iterable[T]) -> T:
    for _ in range(times):
        yield from iterable  # https://www.python.org/dev/peps/pep-0380/

In [145]:
for elem in repeat(5, [1, 2, 3]):
    print(elem, end=' ')

1 2 3 1 2 3 1 2 3 1 2 3 1 2 3 

In [146]:
def repeat(times: int, iterable: T) -> T:
    for _ in range(times):
        yield iterable

In [147]:
for elem in repeat(5, [1, 2, 3]):
    print(elem, end=' ')

[1, 2, 3] [1, 2, 3] [1, 2, 3] [1, 2, 3] [1, 2, 3] 

In [148]:
from collections.abc import Iterator
from dataclasses import dataclass

@dataclass
class BinaryTreeNode:
    value: int
    left: 'BinaryTreeNode | None' = None
    right: 'BinaryTreeNode | None' = None

    def __iter__(self) -> Iterator[int]:
        yield from self.left or ()
        yield self.value
        yield from self.right or ()

In [149]:
tree = BinaryTreeNode(
    left=BinaryTreeNode(
        left=BinaryTreeNode(value=1),
        value=2,
    ),
    value=3,
    right=BinaryTreeNode(
        value=4,
        right=BinaryTreeNode(value=5),
    ),
)

In [150]:
for value in tree:
    print(value, end=' ')

1 2 3 4 5 

### [return в генераторах](https://peps.python.org/pep-0255/#specification-return)

In [151]:
def create_generator() -> Generator[int, None, int]:
    yield 42
    return 21

In [152]:
generator = create_generator()
next(generator)
next(generator)

StopIteration: 21

In [153]:
def generator_wrapper():
    result = yield from create_generator()
    print(result)

In [154]:
list(generator_wrapper())

21


[42]